In [18]:
!pip install ultralytics  # 데이터 셋 분류 YOLO 방식
!pip install torchmetrics # mAP 함수 구하는 메트릭스
!apt-get -qq install fonts-nanum # 시각화시 한글 깨짐으로 추가한 나눔고딕체

zsh:1: command not found: apt-get


### 라이브러리

In [19]:
import matplotlib.font_manager as fm
font_path = '/Users/jeonhyejeong/Library/Fonts/NanumGothic-Bold.ttf'
prop = fm.FontProperties(fname=font_path, size=14)

In [20]:
import os
import json
from glob import glob
from pathlib import Path
from shutil import copy2
from PIL import Image
from collections import defaultdict

import cv2
import random
from ultralytics import YOLO

import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.model_selection import train_test_split
from torchmetrics.detection.mean_ap import MeanAveragePrecision


import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='matplotlib.font_manager')
import logging
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

In [21]:
# 경로 설정 (사용자 맞춤 절대 경로)
ANNOTATIONS_DIR = Path("data/original/train_annotations") # 원본
IMAGES_DIR = Path("data/original/train_images") # 원본
OUTPUT_LABELS  = Path("data/labels") # 수정한 레벨 데이터 경로
OUTPUT_LABELS.mkdir(parents=True, exist_ok=True)

OUTPUT_DIR = Path("data/project_rcnn")
IMAGE_TRAIN = OUTPUT_DIR / "images" / "train" # / img
IMAGE_VAL   = OUTPUT_DIR / "images" / "val"
LABEL_TRAIN = OUTPUT_DIR / "labels" / "train"
LABEL_VAL   = OUTPUT_DIR / "labels" / "val"
VAL_RATIO = 0.2

### 전처리
데이터 분류<br>
기존 코드의 전처리 결과 라벨 파일의 txt 내용이 알약 라벨링 , 바운딩박스 좌표 형태의 1줄 만 존재 <br>
하나의 이미지에 여러 알약이 존재하여 1줄형태가 아닌 이미지에 해당하는 모든 정보 통합으로 변경

In [22]:
for d in [IMAGE_TRAIN, IMAGE_VAL, LABEL_TRAIN, LABEL_VAL]:           # 라벨을 평가데이터와 학습 데이터로 나눔 8 : 2
    d.mkdir(parents=True, exist_ok=True)

# 전체 이미지 목록
all_images = list(IMAGES_DIR.glob("*.png"))
random.shuffle(all_images)

VAL_RATIO = 0.2  # 20% validation
val_cnt = int(len(all_images) * VAL_RATIO)

val_images = set(all_images[:val_cnt])
train_images = set(all_images[val_cnt:])

# 함수: 이미지와 라벨 쌍 복사
def copy_dataset(image_list, img_dst_dir, label_dst_dir):
    for img_path in image_list:
        label_path = OUTPUT_LABELS / (img_path.stem + ".txt")
        copy2(img_path, img_dst_dir / img_path.name)
        if label_path.exists():
            copy2(label_path, label_dst_dir / label_path.name)
        else:
            # 라벨 없는 이미지면 빈 파일로 생성해도 됨 (선택)
            open(label_dst_dir / label_path.name, 'w').close()

copy_dataset(train_images, IMAGE_TRAIN, LABEL_TRAIN)
copy_dataset(val_images, IMAGE_VAL, LABEL_VAL)

print(f"Train images: {len(train_images)} | Val images: {len(val_images)}")

Train images: 1192 | Val images: 297


### 함수 및 변수

In [23]:
id_to_index= {0: 0, 1899: 1, 2482: 2, 3350: 3, 3482: 4, 3543: 5, 3742: 6, 3831: 7, 4377: 8, 4542: 9, 5093: 10, 5885: 11, 6191: 12, 6562: 13, 10220: 14, 12080: 15, 12246: 16, 12419: 17, 12777: 18, 13394: 19, 13899: 20, 16231: 21, 16261: 22, 16547: 23, 16550: 24, 16687: 25, 18109: 26, 18146: 27, 18356: 28, 19231: 29, 19551: 30, 19606: 31, 19860: 32, 20013: 33, 20237: 34, 20876: 35, 21025: 36, 21324: 37, 21770: 38, 22073: 39, 22346: 40, 22361: 41, 22626: 42, 23202: 43, 23222: 44, 24849: 45, 25366: 46, 25437: 47, 25468: 48, 27652: 49, 27732: 50, 27776: 51, 27925: 52, 27992: 53, 28762: 54, 29344: 55, 29450: 56, 29666: 57, 29870: 58, 30307: 59, 31704: 60, 31862: 61, 31884: 62, 32309: 63, 33008: 64, 33207: 65, 33877: 66, 33879: 67, 34596: 68, 35205: 69, 36636: 70, 38161: 71, 41767: 72, 44198: 73}
index_to_name= {0: 'background', 1: '보령부스파정 5mg', 2: '뮤테란캡슐 100mg', 3: '일양하이트린정 2mg', 4: '기넥신에프정(은행엽엑스)(수출용)', 5: '무코스타정(레바미피드)(비매품)', 6: '알드린정', 7: '뉴로메드정(옥시라세탐)', 8: '타이레놀정500mg', 9: '에어탈정(아세클로페낙)', 10: '삼남건조수산화알루미늄겔정', 11: '타이레놀이알서방정(아세트아미노펜)(수출용)', 12: '삐콤씨에프정 618.6mg/병', 13: '조인스정 200mg', 14: '쎄로켈정 100mg', 15: '리렉스펜정 300mg/PTP', 16: '아빌리파이정 10mg', 17: '자이프렉사정 2.5mg', 18: '다보타민큐정 10mg/병', 19: '써스펜8시간이알서방정 650mg', 20: '에빅사정(메만틴염산염)(비매품)', 21: '리피토정 20mg', 22: '크레스토정 20mg', 23: '가바토파정 100mg', 24: '동아가바펜틴정 800mg', 25: '오마코연질캡슐(오메가-3-산에틸에스테르90)', 26: '란스톤엘에프디티정 30mg', 27: '리리카캡슐 150mg', 28: '종근당글리아티린연질캡슐(콜린알포세레이트)\xa0', 29: '콜리네이트연질캡슐 400mg', 30: '트루비타정 60mg/병', 31: '스토가정 10mg', 32: '노바스크정 5mg', 33: '마도파정', 34: '플라빅스정 75mg', 35: '엑스포지정 5/160mg', 36: '펠루비정(펠루비프로펜)', 37: '아토르바정 10mg', 38: '라비에트정 20mg', 39: '리피로우정 20mg', 40: '자누비아정 50mg', 41: '맥시부펜이알정 300mg', 42: '메가파워정 90mg/병', 43: '쿠에타핀정 25mg', 44: '비타비백정 100mg/병', 45: '놀텍정 10mg', 46: '자누메트정 50/850mg', 47: '큐시드정 31.5mg/PTP', 48: '아모잘탄정 5/100mg', 49: '세비카정 10/40mg', 50: '트윈스타정 40/5mg', 51: '카나브정 60mg', 52: '울트라셋이알서방정', 53: '졸로푸트정 100mg', 54: '트라젠타정(리나글립틴)', 55: '비모보정 500/20mg', 56: '레일라정', 57: '리바로정 4mg', 58: '렉사프로정 15mg', 59: '트라젠타듀오정 2.5/850mg', 60: '낙소졸정 500/20mg', 61: '아질렉트정(라사길린메실산염)', 62: '자누메트엑스알서방정 100/1000mg', 63: '글리아타민연질캡슐', 64: '신바로정', 65: '에스원엠프정 20mg', 66: '브린텔릭스정 20mg', 67: '글리틴정(콜린알포세레이트)', 68: '제미메트서방정 50/1000mg', 69: '아토젯정 10/40mg', 70: '로수젯정10/5밀리그램', 71: '로수바미브정 10/20mg', 72: '카발린캡슐 25mg', 73: '케이캡정 50mg', 74: "넥시움정 40mg"}

In [24]:
def show_multiple_predictions(dataset, model, class_names, num_images=12, score_threshold=0.5):  # 이미지 출력 코드 12개의 이미지로 num_images 입력값을 변경해서 이미지를 추가로 더뽑을 수 있음
  idxs = random.sample(range(len(dataset)), k=num_images)
  ncols = 3
  nrows = 4
  fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*5, nrows*5))

  for ax, sample_idx in zip(axes.flatten(), idxs):
      img, _ = dataset[sample_idx]
      with torch.no_grad():
          output = model([img.to(device)])[0]
      npimg = (img.permute(1,2,0).cpu().numpy() * 255).astype(np.uint8).copy()
      ax.imshow(npimg)
      h, w = npimg.shape[:2]
      boxes = output['boxes'].cpu().numpy()
      labels = output['labels'].cpu().numpy()
      scores = output['scores'].cpu().numpy()
      for box, label, score in zip(boxes, labels, scores):
          if score < score_threshold:
              continue
          x1, y1, x2, y2 = map(int, box)
          rect = plt.Rectangle((x1, y1), x2-x1, y2-y1, fill=False, edgecolor='red', linewidth=2)
          ax.add_patch(rect)
          txt = f"{class_names[label]}:{score:.2f}"
          ax.text(x1, y1-5, txt, fontsize=12, color='blue',
                  fontproperties=prop,
                  bbox=dict(facecolor='white', alpha=0.7, boxstyle='round'))
      ax.set_axis_off()
      ax.set_title(f"샘플 {sample_idx}", fontproperties=prop)
  # 빈 ax가 남으면 지우기
  for ax in axes.flatten()[len(idxs):]:
      ax.remove()
  plt.tight_layout()
  plt.show()

In [25]:
def show_test_predictions(dataset, model, class_names, device, num_images=12, score_threshold=0.5): # 테스트 이미지 시각화
    idxs = random.sample(range(len(dataset)), k=min(num_images, len(dataset)))
    ncols = 3
    nrows = int(np.ceil(num_images / ncols))
    fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*5, nrows*5))

    axes = axes.flatten() if num_images > 1 else [axes]

    for ax, sample_idx in zip(axes, idxs):
        img, fname = dataset[sample_idx]
        input_img = img.unsqueeze(0) if img.ndim == 3 else img
        with torch.no_grad():
            output = model(input_img.to(device))[0]
        # img: (C, H, W)
        npimg = (img.permute(1,2,0).cpu().numpy() * 255).astype(np.uint8)
        ax.imshow(npimg)
        boxes = output['boxes'].cpu().numpy()
        labels = output['labels'].cpu().numpy()
        scores = output['scores'].cpu().numpy()
        h, w = npimg.shape[:2]
        for box, label, score in zip(boxes, labels, scores):
            if score < score_threshold:
                continue
            x1, y1, x2, y2 = map(int, box)
            rect = plt.Rectangle((x1, y1), x2-x1, y2-y1, fill=False, edgecolor='red', linewidth=2)
            ax.add_patch(rect)
            # 클래스명이 숫자 리스트인 경우 안전 처리
            label_str = class_names[label] if label < len(class_names) else str(label)
            txt = f"{label_str}:{score:.2f}"
            ax.text(x1, max(y1-5, 0), txt, fontsize=12, color='blue',
                    fontproperties=prop if prop else None,
                    bbox=dict(facecolor='white', alpha=0.7, boxstyle='round'))
        ax.set_axis_off()
        ax.set_title(f"{fname}", fontproperties=prop if prop else None)
    # 남은 빈 ax 지우기
    for ax in axes[len(idxs):]:
        ax.remove()
    plt.tight_layout()
    plt.show()

In [26]:
class LabelDataset(Dataset):                                # 라벨에 여러 데이터가 있는 경우의 데이터셋
    def __init__(self, img_dir, label_dir, transforms=None):
        self.img_paths = sorted(list(Path(img_dir).glob("*.png")))
        self.label_dir = Path(label_dir)
        self.transforms = transforms

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        img = cv2.imread(str(img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w = img.shape[:2]

        # 라벨 읽기
        label_path = self.label_dir / (img_path.stem + ".txt")
        boxes = []
        labels = []
        if label_path.exists():
            with open(label_path, "r", encoding="utf-8") as f:
                for line in f:
                    splits = line.strip().split()
                    if len(splits) != 5:
                        continue
                    class_id = int(splits[0])
                    x_c, y_c, bw, bh = map(float, splits[1:])
                    x1 = (x_c - bw/2) * w
                    y1 = (y_c - bh/2) * h
                    x2 = (x_c + bw/2) * w
                    y2 = (y_c + bh/2) * h
                    boxes.append([x1, y1, x2, y2])
                    labels.append(class_id)
        boxes = torch.tensor(boxes, dtype=torch.float32) if boxes else torch.zeros((0,4), dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64) if labels else torch.zeros((0,), dtype=torch.int64)

        target = {
            "boxes": boxes,
            "labels": labels
        }
        img = torch.from_numpy(img).permute(2,0,1).float() / 255.
        if self.transforms:
            img = self.transforms(img)
        return img, target

In [27]:
class TestImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.image_list = [f for f in os.listdir(image_dir)
                           if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        self.transform = transform

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_list[idx])
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, self.image_list[idx]  # (이미지, 파일명)

### 모델

In [28]:
# # Faster R-CNN 모델 생성 함수
# def get_model_instance(num_classes: int):
#     model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
#     in_features = model.roi_heads.box_predictor.cls_score.in_features
#     model.roi_heads.box_predictor = models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
#     return model

In [29]:
def get_model_instance(num_classes: int):
    model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

    # 모든 backbone 파라미터 freeze
    for name, param in model.backbone.body.named_parameters():
        param.requires_grad = False

    # Layer4만 unfreeze
    for name, param in model.backbone.body.layer4.named_parameters():
        param.requires_grad = True

    # ROI head classifier 변경
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

    return model

In [30]:
# 경로 및 클래스 수 지정
train_img_dir = "data/project_rcnn/images/train"
train_label_dir = "data/project_rcnn/labels/train"
val_img_dir = "data/project_rcnn/images/val"
val_label_dir = "data/project_rcnn/labels/val"
test_img_dir = "data/original/test_images"                   # 원본테스트 이미지 경로

# index_to_name은 {0:'background', 1:'알약1', ...} 형식
num_classes = len(index_to_name)  # background 포함

# 데이터셋, 로더
train_dataset = LabelDataset(train_img_dir, train_label_dir)
val_dataset = LabelDataset(val_img_dir, val_label_dir)
test_dataset = TestImageDataset(test_img_dir, transform= transforms.ToTensor())
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

In [31]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = get_model_instance(num_classes)
model.to(device)
optimizer = torch.optim.AdamW([p for p in model.parameters() if p.requires_grad], lr=1e-3) # 파라미터

/opt/anaconda3/envs/yolov5/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/yolov5/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [32]:
num_epochs = 30

# 학습
model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, targets in train_loader:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)
        loss = sum(loss for loss in loss_dict.values())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

Error: command buffer exited with error status.
	The Metal Performance Shaders operations encoded on it may not have completed.
	Error: 
	(null)
	Internal Error (0000000e:Internal Error)
	<AGXG13GFamilyCommandBuffer: 0x574137020>
    label = <none> 
    device = <AGXG13GDevice: 0x17c3d4e00>
        name = Apple M1 
    commandQueue = <AGXG13GFamilyCommandQueue: 0x17c3ff200>
        label = <none> 
        device = <AGXG13GDevice: 0x17c3d4e00>
            name = Apple M1 
    retainedReferences = 1
Error: command buffer exited with error status.
	The Metal Performance Shaders operations encoded on it may not have completed.
	Error: 
	(null)
	Internal Error (0000000e:Internal Error)
	<AGXG13GFamilyCommandBuffer: 0x5740dedc0>
    label = <none> 
    device = <AGXG13GDevice: 0x17c3d4e00>
        name = Apple M1 
    commandQueue = <AGXG13GFamilyCommandQueue: 0x17c3ff200>
        label = <none> 
        device = <AGXG13GDevice: 0x17c3d4e00>
            name = Apple M1 
    retainedReference

KeyboardInterrupt: 

In [ ]:
# 가중치 저장
torch.save(model.state_dict(), "fasterrcnn_model_R1.pth")

In [ ]:
# 학습된 가중치 불러오기
model.load_state_dict(torch.load("fasterrcnn_model_R1.pth"))
model.to(device)

#### 시각화 및 평가 지표 출력

In [ ]:
# mAP 객체 생성 test 데이터는 라벨이 없어서 출력 불가능
map_metric = MeanAveragePrecision(iou_type="bbox")  # 기본은 COCO mAP@[.5:.95]

model.eval()
device = next(model.parameters()).device
num_images = 200  # 평가할 이미지 개수 (전부 다 할 수도 있음)

all_preds = []
all_targets = []

for i in range(num_images):
    img, target = val_dataset[i]
    with torch.no_grad():
        pred = model([img.to(device)])[0]
    if len(target['labels']) == 0:   # GT 없음
        continue                     # 평가에서 스킵
    # 예측 박스
    preds = {
        "boxes": pred['boxes'].cpu(),
        "scores": pred['scores'].cpu(),
        "labels": pred['labels'].cpu()
    }
    # GT(정답) 박스
    targets = {
        "boxes": target['boxes'],
        "labels": target['labels']
    }
    map_metric.update([preds], [targets])  # 리스트 형태여야 함

# mAP, AP50 등 리포트
result = map_metric.compute()
print("mAP:", result['map'].item()) # 50~95
print("mAP@0.5:", result['map_50'].item())
print("mAP@0.75:", result['map_75'].item())

In [ ]:
model.eval()
class_names = [index_to_name[i] for i in range(len(index_to_name))]
show_multiple_predictions(val_dataset, model, class_names, num_images=10, score_threshold=0.5) # val_data로 시각화

In [ ]:
model.eval()
class_names = [index_to_name[i] for i in range(len(index_to_name))]
show_test_predictions(test_dataset, model, class_names, device, num_images=12, score_threshold=0.5) # test_data로 시각화